In [ ]:
import numpy as np 

# Initialize parameters 
n_fft = 4096     # fft length
fs = 44100       # sampling rate
n_hop = 256      # window overlap 256 - 1024 
n_templates = 30 # number of notes in library 
template_labels = n_templates*[None]


In [ ]:
# Writing NMF for polyphonic music analysis 

# Read Template data

# Create Template Matrix W
    # Each column is an averaged FFT over the note's sustain 
    


In [2]:
# beta-divergence cost function 
# @args
# v - input sound vector 
# W - template library 
# h - activation vector 

# calculate cost by interpolating between square euclidian and 
def beta_divergence (v, W, h): 
    estimate = np.dot(W,h)
    # half euclidean distance (beta = 2)
    half_euc = np.sum(0.5*(v - estimate)**2)
    # kullback-liebler distance (beta = 1)
    k_liebler = np.sum(v*np.log(v/estimate) + v - estimate)
    
    print ("half_euclidean", half_euc)
    print ("k_liebler", k_liebler)
    return 0.5 * (half_euc + k_liebler)
    
    
    
# update activation matrix, h is updated in place
def update_estimate (v, W, h, beta):
    estimate = np.dot(W,h)
    v_matrix = np.outer(v, np.ones(np.shape(h,0)))
    numerator = np.dot( (W * v_matrix).T, estimate** (beta - 2))
    denominator = np.dot( W.T, estimate**(beta-1) )
    h *= (numerator/denominator)
    

# Iteratively update h until cost < absolute threshold and 
def iterate (cost_threshold, activation_threshold, v, W, h, beta):
    cost = float('inf')
    while cost > cost_threshold:
        update_estimate(v,W,h,beta)
        cost = beta_divergence(v,W,h)
        
    notes = template_labels[h > activation_threshold]
    return notes 

        
    
                       

SyntaxError: unexpected EOF while parsing (<ipython-input-2-ed69032ac9f3>, line 33)